In [1]:
from lda_for_fragments import *

In [2]:
n_topics = 50

In [3]:
# beer data with binary matrix
run_lda('beer_pos', 'input/QC_Beer_ParentsFragments_Tonyscripting.csv', True, n_topics)

Binary input from input/QC_Beer_ParentsFragments_Tonyscripting.csv
Data shape (511, 950)
Fitting model...
DONE!
Writing topics to results/beer_pos/binary/beer_pos_binary_topics.csv
Writing fragments x topics probability matrix to results/beer_pos/binary/beer_pos_binary_all.csv
Writing topic docs to results/beer_pos/binary/beer_pos_binary_docs.csv


In [4]:
# beer data with multinomial matrix
run_lda('beer_pos', 'input/QC_Beer_ParentsFragments_Tonyscripting.csv', False, n_topics)

Multinomial input from input/QC_Beer_ParentsFragments_Tonyscripting.csv
Data shape (511, 950)
Fitting model...
DONE!
Writing topics to results/beer_pos/multinomial/beer_pos_multinomial_topics.csv
Writing fragments x topics probability matrix to results/beer_pos/multinomial/beer_pos_multinomial_all.csv
Writing topic docs to results/beer_pos/multinomial/beer_pos_multinomial_docs.csv
